In [84]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [2]:
URL = 'https://www.bybt.com/Grayscale'
HEADERS = {'accept':
           'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,\
           image/webp,image/apng,*/*;q=0.8,application/signed-exchange;\
           v=b3;q=0.9', 'user-agent':
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, \
           like Gecko) Chrome/86.0.4240.111 Safari/537.36'}

In [3]:
def get_html(url, params=None):
    return requests.get(url, headers=HEADERS, params=params)


In [15]:
html = get_html(URL)
soup = BeautifulSoup(html.text, 'html.parser')
items = soup.find('div', class_='ivu-table ivu-table-default')

In [27]:
headers = [i.text.strip() for i in items.find_all('th')]

In [45]:
body = items.find_all('tbody')[0].find_all('tr')
rows = [i.text.split() for i in [b for b in body]]

In [53]:
def mutation_indexes(data):
    data = data[:]
    if len(data) == 14:
        del data[1]
        del data[12]
    if len(data) == 13:
        del data[12]
    return data


In [76]:
json_file = [mutation_indexes(i) for i in rows]
df  = pd.DataFrame(json_file)
df = df.drop([2], axis=1)
df.columns = headers
df.head()

,Symbol,Total Holdings Total Holdings,Total Holdings($),Holdings(Per Share),Marker Price(Per Share),Premium,Close Time,24H Change,7D Change,30D Change,Update Time
0,BTC,649.78K,$30.61B,$42.82,$38.34,-10.46%,2021-08-14,+0,-249,-1034,2021-08-15
1,ETH,3.14M,$10.29B,$31.97,$30.46,-4.72%,2021-08-14,+0,-1508,-6252,2021-08-15
2,ETC,12.3M,$823.29M,$53.52,$33.38,-37.63%,2021-08-14,+0,-7077,-29346,2021-08-15
3,LTC,1.58M,$287.89M,$16.07,$16.95,5.48%,2021-08-14,+0,-757,+32954,2021-08-15
4,BCH,319.17K,$219.49M,$5.74,$5.64,-1.74%,2021-08-14,+0,-153,+21066,2021-08-15


# Trips to parse data

In [78]:
# Open interest BTC from Grey Scal
data = requests.get('https://fapi.bybt.com/api/grayscaleOpenInterest/history?symbol=BTC', headers=HEADERS)

In [92]:
open_interest_GS = pd.DataFrame(json.loads(data.text).get('data'))
open_interest_GS['dateList'] = pd.to_datetime(open_interest_GS['dateList'])

In [95]:
data = requests.get('https://fapi.bybt.com/api/grayscale/market/history?symbol=BTC', headers=HEADERS)
historical_data = pd.DataFrame(json.loads(data.text).get('data'))

In [97]:
historical_data.tail(10)

,markerPriceList,dateList,holdingsList,premiumRateList
1976,33.56,1627938000000,37.40,-10.27
1977,31.88,1628024400000,36.16,-11.84
1978,33.14,1628110800000,36.25,-8.58
1979,33.70,1628197200000,36.97,-8.85
1980,36.00,1628283600000,38.93,-7.53
1981,38.55,1628542800000,41.93,-8.06
1982,37.80,1628629200000,42.82,-11.72
1983,38.87,1628715600000,43.24,-10.11
1984,36.86,1628802000000,42.29,-12.84
1985,38.34,1628888400000,42.82,-10.46


# Parse data from
1. https://www.bybt.com/BitcoinOpenInterest

In [98]:
html = get_html('https://www.cftc.gov/dea/futures/financial_lf.htm')
soup = BeautifulSoup(html.text, 'html.parser')
# https://stackoverflow.com/questions/49949849/how-to-parse-the-data-in-pre-tag-using-beautifulsoup

In [99]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en-US" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>CFTC Commitments of Traders Short Report - Financial Traders in Markets (Futures Only)</title>
<!--begin meta-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="0" http-equiv="Expires"/>
<meta content="This is the viewable version of the most recent release of the Financial Traders in Markets short form Futures Only commitments report." name="description">
<meta content="finacial_lf.htm" name="file">
<meta content="commitments, short, CFTC, Finacial Traders in Markets, Futures Only" name="keywords"/>
<meta content="dea" name="office"/>
<!--end meta-->
<script src="http://www.google-analytics.com/ga.js" type="text/javascript"></script><script type="text/javascript">var pageTracker = _gat._getTracker("